이 파일은 item x item 상호작용 matrix를 cluster x item 상호작용 matrix로 계산 및 변환하는 파일입니다!!

In [26]:
import pandas as pd
import numpy as np
import openpyxl

from collections import Counter
from itertools import permutations
from tqdm import tqdm

In [27]:
data_path = '/opt/ml/input/data/raw_codishop/view/item/item_codi_id.xlsx'
preprocessed_path = '/opt/ml/input/data/asset_codishop/view/item/item.csv'
# preprocessed_path = './temp_color_midclass.xlsx'

item_df = pd.read_excel(data_path, engine='openpyxl')
preprocessed_df = pd.read_csv(preprocessed_path)

In [28]:
drop_indexes = list()
for i in range(len(item_df)):
    if item_df.iloc[i]["id"] not in preprocessed_df["id"].unique() :
        drop_indexes.append(i)

item_df = item_df.drop(drop_indexes)

item x item 상호작용 행렬을 생성하는 파일

In [29]:
items_in_codi = item_df.groupby('codi_id')['id'].apply(list)

In [30]:
# 코디 내에서 가능한 아이템들의 조합을 계산하는 코드
possible_combinations = list()
items_in_codi = item_df.groupby('codi_id')['id'].apply(list)

for val in tqdm(items_in_codi):
    val = sorted(val)
    comb_res = list(permutations(val, 2))
    possible_combinations.extend(comb_res)

100%|██████████| 2300/2300 [00:00<00:00, 336715.50it/s]


In [31]:
pivot_list = list()
for item1, item2 in possible_combinations:
    cluster_id = preprocessed_df[preprocessed_df['id'] == item1]['cluster_id'].values
    pivot_list.append((cluster_id[0], item2))

In [32]:
count_combination = list(Counter(pivot_list).most_common())

In [33]:
result = list()
for (cluster_id, item_id), cnt in count_combination:
    result.append((cluster_id, item_id, cnt))

In [34]:
IIM = pd.DataFrame(result, columns=["id", "item", "count"]).pivot_table(index="id", columns="item", values="count").fillna(0)
IIM = IIM.astype(int)
# TODO : 데이터 경로 설정
IIM.to_csv("/opt/ml/input/data/asset_codishop/view/item/itemInteractionMatrix_withColor.csv", index=True)

In [35]:
IIM

item,67485,76867,102620,102621,104238,104239,104240,104241,108891,110640,...,2460711,2460717,2462470,2462500,2464514,2471391,2474376,2476116,2486641,2500338
id,,,,,,,,,,,,,,,,,,,,,
0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26233,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26285,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26302,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [36]:
ones = 0
zeros = 0
for i in range(len(IIM)) :
    oz_cnt = IIM.iloc[i].value_counts()
    zeros += oz_cnt[0]
    ones += sum(oz_cnt[1:])
print(f"Sparsity of this Data is : {zeros/(ones+zeros):.4}")

Sparsity of this Data is : 0.9981


In [43]:
total_sum, total_cnt = 0, 0
max_val = 0
for idx, row in IIM.iterrows():
    max_val = max(max_val, max(row))
    total_sum += sum(row)
    total_cnt += len(row)
    # break

print (max_val)
print (total_sum, total_cnt, total_sum / total_cnt)

4
22202 11736373 0.001891725833867073


In [38]:
sum(IIM) / (len(IIM.columns) * len(IIM.index))

670.7832568886487